<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>

### 3.2 Model Building with Traffic
---


<div style="text-align: justify; line-height: 2">

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Having observed that weather conditions do not have a significant impact on the end delay of STM buses, we will attempt to build a model that also includes traffic data to see if it can improve the model's predictive ability. The new model will include the same weather and bus data as the previous unfiltered model. This data will be combined with the traffic data from the City of Montreal4 . [See master + traffic dataset description](../Data/Dataset_descriptions.md). A sample of the data is shown below.

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|     | line | delay_start | start_time | temp_start | rain_amt | snow_amt | delay_end | total_traffic |
| :-: | ---: | ----------: | ---------: | ---------: | -------: | -------: | --------: | ------------: |
|  0  |  467 |           0 |        405 |        7.1 |        0 |        0 |         6 |          3674 |
|  1  |  467 |           1 |        437 |        6.9 |        0 |        0 |         5 |          3116 |
|  2  |  467 |           1 |        468 |        6.9 |        0 |        0 |         6 |          3984 |
|  3  |  467 |          -1 |        496 |        6.5 |        0 |        0 |         6 |          2655 |
|  4  |  467 |           0 |        527 |        6.5 |        0 |        0 |         6 |          2273 |

|     | T_NBLT | T_NBT | T_NBRT | T_SBLT | T_SBT | T_SBRT | T_EBLT | T_EBT | T_EBRT | T_WBLT | T_WBT | T_WBRT |
| :-: | -----: | ----: | -----: | -----: | ----: | -----: | -----: | ----: | -----: | -----: | ----: | -----: |
|  0  |      0 |  1236 |     23 |      4 |  1270 |    155 |    165 |   157 |     68 |      2 |   478 |    116 |
|  1  |      0 |  1031 |     56 |      0 |   927 |    103 |    130 |   238 |     45 |      0 |   473 |    113 |
|  2  |      0 |  1330 |     44 |      0 |  1167 |    170 |    191 |   312 |     89 |      3 |   567 |    111 |
|  3  |      0 |   853 |     35 |      0 |   884 |    113 |    116 |   171 |     86 |      1 |   321 |     75 |
|  4  |      0 |   762 |     32 |      0 |   774 |     81 |     96 |   164 |     47 |      1 |   253 |     63 |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.1: Sample of the data used for model building with traffic.</p>

</td>
</tr>
</table>

<table align="center" style="border: none;">
<tr>
<td style="text-align: center; border: none;">

|   Column Name   |                                    Description                                    |
| :-------------: | :-------------------------------------------------------------------------------: |
|     *line*      |                                  Bus line number                                  |
|  *delay_start*  |                    Delay at the start of the trip (in minutes)                    |
|  *start_time*   |         Time of day at the start of the trip (in minutes after midnight)          |
|  *temp_start*   |             Temperature at the start of the trip (in degrees Celsius)             |
|   *rain_amt*    |                   Rain amount at the start of the trip (in cm)                    |
|   *snow_amt*    |                   Snow amount at the start of the trip (in cm)                    |
|   *delay_end*   | Delay at the end of the trip (in minutes, negative values indicate early arrival) |
| *total_traffic* |               Total traffic during the trip (in number of vehicles)               |

<p style="font-weight: bold; font-style: italic; text-align: center;">Table 3.2.2: Description of the columns in the data set used for model building with traffic.</p>

</td>
</tr>
</table>

The coded columns represent the number of vehicles with a certain direction and turn at all the intersections during the trip. The first two letters represent the direction bound (N, S, E, W), and the last letters represent the turn (LT = left turn, T = through, RT = right turn). For example, T_NBLT represents the number of vehicles going through the intersection in the northbound direction and turning left. The total traffic is the sum of all the coded columns.











</div>
&nbsp;  
&nbsp;  

---

<a name="footnote1"></a>1: [PennState Eberly College of Science](./5_References.ipynb#1)  
<a name="footnote2"></a>2: [Ivan T. Ivanov](./5_References.ipynb#2)  
<a name="footnote3"></a>3: [OpenStax](./5_References.ipynb#3)  
<a name="footnote4"></a>4: [Heij, De Boer; Franses, Kloep; and Van Dijk](./5_References.ipynb#4)  
<a name="footnote5"></a>5: [Mostoufi, Navid; Constantinides, Alkis](./5_References.ipynb#5)

<div style="display: flex; justify-content: space-between;">
<a style="flex: 1; text-align: left;" href="./3_1_MLR_bus_weather.ipynb">← Previous: 3.1 Model Building with Weather</a>
<span style="flex: 1; text-align: center;">3.2 Model Building with Traffic</span>
<a style="flex: 1; text-align: right;" href="./4_Conclusions.ipynb">Next: Conclusions →</a>
</div>
